# Scraped Fixture API

- Get your personal Signal Ocean API subscription key (acquired [here](https://apis.signalocean.com/profile)) and replace it below:

In [1]:
signal_ocean_api_key = '5ee27a085a1a40ac8cc84309cab3957e' # replace with your subscription key

## Setup

### Import libraries

In [2]:
!pip install signal-ocean

import pandas as pd
from datetime import date, timedelta
import plotly.graph_objects as go
import plotly.express as px


from signal_ocean import Connection
from signal_ocean.scraped_fixtures import ScrapedFixturesAPI

You should consider upgrading via the 'c:\users\g.mema\signal_tools\programming_language\python\python38\python.exe -m pip install --upgrade pip' command.


### set up parameters

In [12]:
vessel_type_id = 1 # Tanker
period_in_days = 7
vessel_class_id = 84 # VLCC
port_id = 3778 # Ras Tanura

## Scraped Fixtures 

### Get Scraped Fixtures using ScrapedFixturesAPI

In [13]:
def get_scraped_fixtures():
    date_from = date.today() - timedelta(days=period_in_days)
    date_to = date.today()

    # Scraped Fixture API
    connection = Connection(signal_ocean_api_key)
    api = ScrapedFixturesAPI(connection)
    scraped_fixtures = api.get_fixtures(
        received_date_from = date_from,
        received_date_to = date_to,
        vessel_type = vessel_type_id,
        port_id = port_id,
        vessel_class_id = vessel_class_id
    )
    
    data = pd.DataFrame([v.__dict__ for v in scraped_fixtures])
    return data

In [26]:
def plot_charterers(data, top_k_charterers=3, days = 7):

  data_aggr = data \
              .groupby(['charterer'])['imo'] \
              .unique().reset_index()

  data_aggr['Count Unique Imo'] = data_aggr \
                                            .apply(lambda x: len(x.imo),axis=1)
  top_k = data_aggr \
          .sort_values('Count Unique Imo') \
          .tail(top_k_charterers)
  
  fig = px.bar(top_k, 
               y='charterer', 
               x='Count Unique Imo' ,
               orientation='h',
               title="Top {} charterers, last {} days, VLCC" \
                              .format(top_k_charterers, days)
               )
    
  
  fig.update_layout(
      xaxis_title="Number of fixed Vessels",
      yaxis_title="Charterers"
  )
  fig.show()

In [32]:
def plot_weekly_fixed_vessels(results):
  freq_dict = {
      0:'W-MON',
      1:'W-TUE',
      2:'W-WED',
      3:'W-THU',
      4:'W-FRI',
      5:'W-SAT',
      6:'W-SUN'
  }
  fixtures_weekly_laycan = results.groupby(
      pd.Grouper(
          key='laycan_from',
          freq = freq_dict[date.today().weekday()],
          closed = 'left',
          label = 'left' 
      )
  )['imo'].unique().reset_index()
  fixtures_weekly_laycan['Count Unique Imo'] = fixtures_weekly_laycan \
                                            .apply(lambda x: len(x.imo),axis=1)

  fig = px.line(
      fixtures_weekly_laycan, 
      title="Port Arrivals per Week",
      x='laycan_from', 
      y='Count Unique Imo', 
      
      # hover_data={"laycan_from": "|%a %d-%m-%Y"},
      # line_dash = 'count distinct IMO'
      )
  fig.update_layout(
      yaxis_title="Number of fixed Vessels",
      xaxis_title="Laycan Date"
  )
  fig.show()

## Results

### Get fixtures for last 30 days Ras Tanura/VLCC

In [22]:
%%time
results = get_scraped_fixtures()
results.head(5)

Wall time: 1min 6s


,fixture_id,message_id,updated_date,received_date,reported_fixture_date,imo,vessel_class_id,laycan_from,laycan_to,load_geo_id,...,redelivery_from_taxonomy,redelivery_to_name,redelivery_to_taxonomy,charterer_type,fixture_status,content,sender,is_private,is_invalidated,is_partial
0,83757071,22528158,2022-03-17 05:00:07+00:00,2022-03-17 04:54:46+00:00,None,9563237,84,2022-03-18 00:00:00+00:00,2022-03-18 00:00:00+00:00,3778,...,None,None,None,None,Failed,seaways kilimanjaro 12 280 18/03 rt/ecc cc w26...,Howerob,True,False,False
1,83757077,22528158,2022-03-17 05:00:07+00:00,2022-03-17 04:54:46+00:00,None,9693745,84,2022-03-15 00:00:00+00:00,2022-03-16 00:00:00+00:00,3778,...,None,None,None,None,None,miltiadis junior 14 270 15-16/03 rt/china rnr ...,Howerob,True,False,False
2,83757107,22528158,2022-03-17 05:00:07+00:00,2022-03-17 04:54:46+00:00,None,9875783,84,2022-03-25 00:00:00+00:00,2022-03-27 00:00:00+00:00,3778,...,None,None,None,None,None,cssc liao ning 20 270 25-27/03 rt/onsan own pr...,Howerob,True,False,False
3,83772506,22538583,2022-03-17 09:47:47+00:00,2022-03-17 09:47:55+00:00,None,9875783,84,2022-03-25 00:00:00+00:00,2022-03-25 00:00:00+00:00,3778,...,None,None,None,None,None,cssc liao ning 270 25/03 ras tanura/onsan own ...,Telemark Chartering,True,False,False
4,83782744,22541859,2022-03-17 11:21:22+00:00,2022-03-17 11:20:37+00:00,None,9875783,84,2022-03-25 00:00:00+00:00,2022-03-25 00:00:00+00:00,3778,...,None,None,None,None,None,cssc liao ning 270 25/03 rt/onsan o/p koch,SSY,True,False,False


### Plot top k charterers

In [27]:
plot_charterers(results, top_k_charterers=3, days=period_in_days)

### Get the weekly number of fixed vessels for Ras Tanura/VLCC

In [33]:
plot_weekly_fixed_vessels(results)